In [1]:
from urllib import request
import re
import pandas as pd
import urllib.parse as urp
import requests

import csv

In [2]:
def __get_location1__(name,city):  
        my_ak = 'uXuYBYbI90SdKOGSjIqgDnthz6DrkFN4'   
        qurey = urp.quote(name)
        try:
            url = 'http://api.map.baidu.com/place/v2/search?query='+qurey+'&tag='+'&region='+urp.quote(city)+'&output=json&ak='+my_ak
            #print(url)
            req = request.urlopen(url)
            res = req.read().decode()
            lat = pd.to_numeric(re.findall('"lat":(.*)',res)[0].split(',')[0])
            lng = pd.to_numeric(re.findall('"lng":(.*)',res)[0])
            return (lng,lat)  #经度和纬度
        except:
            return 0,0

In [4]:
print(__get_location1__('静安静安寺华山路421弄','上海'))

(121.45185900000001, 31.22366)


In [5]:
df = pd.read_excel('data_after_clean.xls')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df['经度']=0
df['纬度']=0
df.head()

,小区,价格,面积,室,厅,卫,楼层,近地铁,随时看房,精装,经度,纬度
0,山西小区,6700,65.00,2.0,0,NaN,NaN,1.0,NaN,NaN,0,0
1,明园森林都市滴翠苑,8500,91.00,2.0,2,NaN,NaN,1.0,NaN,NaN,0,0
2,四元路20弄小区,2600,56.07,2.0,1,NaN,NaN,0.0,NaN,NaN,0,0
3,明园森林都市滴翠苑,5500,84.00,2.0,2,NaN,NaN,1.0,NaN,NaN,0,0
4,紫华佳苑,6800,78.00,1.0,2,NaN,NaN,1.0,NaN,NaN,0,0


In [6]:
location = df['小区']
lng = []
lat = []

for loc in location:
    address=__get_location1__(str(loc),'上海')
    
    while address == (0,0):
        for i in range(5):
            address = __get_location1__(str(loc),'上海')
            if address ==(0,0):
                continue
            else:
                break
        break
    
    lng.append(str(address[0]))
    lat.append(str(address[1]))

df['经度']=lng
df['纬度']=lat

df.head()

,小区,价格,面积,室,厅,卫,楼层,近地铁,随时看房,精装,经度,纬度
0,山西小区,6700,65.00,2.0,0,NaN,NaN,1.0,NaN,NaN,121.489484,31.240372999999998
1,明园森林都市滴翠苑,8500,91.00,2.0,2,NaN,NaN,1.0,NaN,NaN,121.46085500000001,31.295211
2,四元路20弄小区,2600,56.07,2.0,1,NaN,NaN,0.0,NaN,NaN,121.44033799999998,31.419139
3,明园森林都市滴翠苑,5500,84.00,2.0,2,NaN,NaN,1.0,NaN,NaN,121.46085500000001,31.295211
4,紫华佳苑,6800,78.00,1.0,2,NaN,NaN,1.0,NaN,NaN,121.543922,31.278916


In [8]:
df = df[['价格','面积','经度','纬度','室','厅','卫','楼层','近地铁','精装','随时看房','小区']]
df = df.rename(columns = {'价格': 'Price','面积':'Area', '室':'Bedrooms', '厅':'Livingrooms', '卫':'Bathrooms', '楼层':'Floor', '近地铁':'NextToSubway', '随时看房':'OpenForVisits', '精装':'Exquisite', '经度': 'Longitude', '纬度': 'Latitude'}, inplace = False)
df=df.drop(columns=["小区"])
df.to_excel('data_after_cleaning_new.xls')